Import needed library  
torch.nn -> Parent object for PyTorch models  
torch.nn.functional -> Activation function

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

LeNet-5
![title](images/image_02.png)

One of the earliest CNN's. It was build to read small images of handwritten numbers and correctly classify which digit is represented in the image.

Here’s the abridged version of how it works:
- Layer C1 is a convolutional layer, meaning that it scans the input image for features it learned during training. It outputs a map of where it saw each of its learned features in the image. This “activation map” is downsampled in layer S2.
- Layer C3 is another convolutional layer, this time scanning C1’s activation map for combinations of features. It also puts out an activation map describing the spatial locations of these feature combinations, which is downsampled in layer S4.
- Finally, the fully-connected layers at the end, F5, F6, and OUTPUT, are a classifier that takes the final activation map, and classifies it into one of ten bins representing the 10 digits.

This CNN is represented with the following code:

In [2]:
class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        
        # 1 input image channel (B&W), 6 output channels, 5x5 square convolution kernel
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        
        # An affine operation: y = Wx + b
        self.fc1 = nn.Linear(16 * 5 * 5, 120)   # 5 * 5 from image dimensions
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
    
    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        
        # If the size is square only a single number can be specified
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        
        return x
    
    def num_flat_features(self, x):
        # All dimensions except the batch dimensions
        size = x.size()[1:]
        num_features = 1
        
        for s in size:
            num_features *= s
        
        return num_features

In [3]:
net = LeNet()
print(net)

# Stand-in for 32x32 B&W image
input_stand_in_image = torch.rand(1, 1, 32, 32)

print(f"\nImage batch shape:\n{input_stand_in_image.shape}")

# Output
# forward() is not called directly
output = net(input_stand_in_image)
print(f"\nRaw output:\n{output}")
print(output.shape)

LeNet(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)

Image batch shape:
torch.Size([1, 1, 32, 32])

Raw output:
tensor([[-0.1030,  0.0809,  0.1457,  0.1024,  0.1265, -0.0276,  0.0549,  0.0919,
          0.0300,  0.0299]], grad_fn=<AddmmBackward0>)
torch.Size([1, 10])
